In [ ]:
import warnings
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Metatrader_WINN.csv', parse_dates=['datetime'])
# Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$N'], TFRAMES=['M1','M5','M15','H1'])
# Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src[['A','Z']]                      = Query[['a','z']]
    Src[['Symbol','TF']]                = Query[['symbol','tf']]

    Src['Datetime']                     = Query['datetime']
    Src['Date']                         = Query['datetime'].dt.date
    Src['Time']                         = Query['datetime'].dt.time

    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]

    Src['Change']       = (Src['Close'] - Src['Open'])
    Src['Hilo']         = (Src['High']  - Src['Low'])

    Src['Sign Bull']    = Src['Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Src['Sign Bear']    = Src['Change'].apply(lambda x: -1 if (x <  0) else nan)
    return Src 

Src = FORMAT_SOURCE(Query)
# Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['T'] = Calc.groupby(['Symbol','TF','Date'], sort=0)['Time'].rank()

    Calc['Chg Abs'] = Calc['Change'].abs()
    Calc['Chg Pos'] = Calc['Chg Abs'] * Calc['Sign Bull']
    Calc['Chg Neg'] = Calc['Chg Abs'] * Calc['Sign Bear']

    Calc['HL Pos']  = Calc['Hilo'] * Calc['Sign Bull']
    Calc['HL Neg']  = Calc['Hilo'] * Calc['Sign Bear']
    return Calc

Calc = CALCULATIONS(Src)
# Calc.head()

# Stats

In [28]:
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['Hilo']].describe().round(0).astype(int)

Hilo                                    
    count mean  std  min  25%  50%  75%   max
TF                                           
M5    905  415  172  130  295  385  485  1745
M15  1245  519  222  170  365  470  615  2470
H1   1245  746  332  215  505  670  905  2965

In [29]:
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['HL Pos']].describe().round(0).astype(int)

HL Pos                                    
     count mean  std  min  25%  50%  75%   max
TF                                            
M5     444  412  155  165  300  385  485  1100
M15    606  520  220  170  365  475  620  1895
H1     637  737  317  215  500  670  895  2355

In [30]:
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['HL Neg']].describe().round(0).astype(int)

HL Neg                                    
     count mean  std   min  25%  50%  75%  max
TF                                            
M5     461 -417  188 -1745 -485 -380 -295 -130
M15    639 -517  225 -2470 -610 -465 -365 -175
H1     608 -756  348 -2965 -915 -670 -510 -275

In [31]:
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['Change']].describe().round(0).astype(int)

Change                                    
     count mean  std   min  25% 50%  75%   max
TF                                            
M5     905  -10  266 -1640 -185 -10  165   795
M15   1245   -8  330 -1450 -210 -10  185  1130
H1    1245   -8  480 -2355 -295  10  280  2045

In [32]:
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['Chg Abs']].describe().round(0).astype(int)

Chg Abs                                   
      count mean  std min  25%  50%  75%   max
TF                                            
M5      905  209  164   0   80  175  295  1640
M15    1245  253  211   0   90  200  360  1450
H1     1245  365  312   0  135  290  515  2355

In [33]:
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['Chg Pos']].describe().round(0).astype(int)

Chg Pos                                   
      count mean  std min  25%  50%  75%   max
TF                                            
M5      444  203  152   0   85  170  281   795
M15     606  252  212   0   90  190  360  1130
H1      637  349  301   0  115  270  505  2045

In [34]:
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['Chg Neg']].describe().round(0).astype(int)

Chg Neg                                   
      count mean  std   min  25%  50%  75% max
TF                                            
M5      461 -215  175 -1640 -310 -180  -80  -5
M15     639 -255  210 -1450 -365 -205  -90  -5
H1      608 -382  323 -2355 -530 -302 -150  -5

# Snippets

In [35]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src['Change']       = (Src['Close'] - Src['Open'])
    Src['Hilo']         = (Src['High']  - Src['Low'])
    Src['Sign Bull']    = Src['Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Src['Sign Bear']    = Src['Change'].apply(lambda x: -1 if (x <  0) else nan)
    return Src 


def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['T'] = Calc.groupby(['Symbol','TF','Date'], sort=0)['Time'].rank()

    Calc['Chg Abs'] = Calc['Change'].abs()
    Calc['Chg Pos'] = Calc['Chg Abs'] * Calc['Sign Bull']
    Calc['Chg Neg'] = Calc['Chg Abs'] * Calc['Sign Bear']

    Calc['HL Pos']  = Calc['Hilo'] * Calc['Sign Bull']
    Calc['HL Neg']  = Calc['Hilo'] * Calc['Sign Bear']
    return Calc

In [36]:
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['Hilo']].describe().round(0).astype(int)
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['HL Pos']].describe().round(0).astype(int)
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['HL Neg']].describe().round(0).astype(int)

Calc[Calc['T']==1].groupby(['TF'], sort=0)[['Change']].describe().round(0).astype(int)
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['Chg Abs']].describe().round(0).astype(int)
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['Chg Pos']].describe().round(0).astype(int)
Calc[Calc['T']==1].groupby(['TF'], sort=0)[['Chg Neg']].describe().round(0).astype(int)

Chg Neg                                   
      count mean  std   min  25%  50%  75% max
TF                                            
M5      461 -215  175 -1640 -310 -180  -80  -5
M15     639 -255  210 -1450 -365 -205  -90  -5
H1      608 -382  323 -2355 -530 -302 -150  -5